<img src='./images/Rectangle.svg' width=400 height=400/>


**<u>Mo</u>**lecular **<u>S</u>**imulation **<u>De</u>**sign **<u>F</u>**ramework, or MoSDeF, is a robust Python-based, open-source software framework designed to faciliate the initialization, atom-typing, and screening of soft matter systems using molecular simulation. The project was initially developed at Vanderbilt University, in collaboration with software engineers from the Institute for Software Integrated Systems (ISIS). The project has now expanded into a multi-university collaboration with Vanderbilt University as the lead institution. 


The MoSDeF software suite comprises three main libraries: `mBuild`, `Foyer`, and `GMSO`.  Each library targets a different step of system initialization. Specifically, the `mBuild` library can be used to systematically construct any molecular system, including atomistic and coarse-grained, while `Foyer` can be used to assign interaction parameters to all particles, bonds, angles, and dihedrals in the created system. The `GMSO` library, currently under development, will serve as the main data structure that can be used to store all the information of the system, including the details of the system (particles and their positions) along with types and parameters of that system. 


The MoSDeF suite of tools allows users to build complicated systems in a manner that is scriptable and automated.   This enables molecular simulation studies to be more transparent and reproducible, as these scripts can be easily shared on software hosting services, such as GitHub.  Automation also allows for rapid system initialization for large-scale screening studies.

## Notebook Overview

In this notebook, we will demonstrate how MoSDeF can be used in a molecular simulation workflow. Along the way we will highlight important features of each package. In context of a short demonstration, we will run a simple simulation (equilibrium simulation of a box of small molecules). At the end of the notebook, examples of more complicated example/tutorials are included, which were inspried by previously published work.

For demonstration purposes, we will be using iPy Widgets to show off the different functionalities of MoSDeF.  Note that typical system initialization workflows using MoSDeF are entirely written in Python and do not involve the use of Widgets. Once any of the widgets is activated, it will also print the script of what is happening under the hood.

### mBuild - https://github.com/mosdef-hub/mbuild  

`mBuild` is the molecule building library within MoSDeF. This library includes features that allow users to create complex molecular systems starting from the particle level. The core data structure in `mBuild` is the `Compound` class, which can be used to represent anything from a single particle to a complete system with ten of thoudsands of particles.  
<img src='images/pmpc.png' width=800 height=800/>
With the hierarchial (nestable), component based structure design, users can switch out components to create new systems. The code to create these systems can also be organized into separate Python packages, which we call `mBuild Recipes`.

`mBuild` allows the initialization of `Compound` objects in several different ways.  Here we demonstrate the functionality of loading a `Compound` from the 2D representation of a molecular structure known as SMILES (Simplified Molecular Input Line Entry System) string.

In [1]:
import mbuild as mb
# Loading a compound from a SMILES string
compound = mb.load('C1CCCCC1', smiles=True)
compound.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Initialization of a molecule from a SMILES string is a handy tool to quickly create small molecules, since they can be easily found in a public database, such as https://pubchem.ncbi.nlm.nih.gov/ 


In the demonstration below, you can try to enter a SMILES string, and the `mbuild` library will read it in, turn it into a `mbuild.Compound` and visualize it with `py3dmol`! 


If you don't have one in mind, try:
- Caffeine: `CN1C=NC2=C1C(=O)N(C(=O)N2C)C`
- Nicotine: `CN1CCCC1C2=CN=CC=C2`
- Cannabidiol: `CCCCCC1=CC(=C(C(=C1)O)C2C=C(CCC2C(=C)C)C)O`

In [3]:
import src.widget_events as widgets
import ipywidgets as iwidgets

display(widgets.smiles_box)
display(widgets.out_smiles)

Text(value='', description='SMILES:', placeholder='Enter a SMILES string', style=DescriptionStyle(description_…

Output()

`mbuild.load()` can easily load in a wide range of common molecule file formats, such as `pdb`, `mol2`, `xyz`, etc.

In this example workflow, we will demonstrate the process of loading small molecules from `mol2` files located in this repo to build a box of molecules, where users can specify the size of the box and the number of particles in the box. 

In [ ]:
display(widgets.compound_dropdown)
display(widgets.out_mol)

In [ ]:
display(iwidgets.VBox([widgets.box_slider, widgets.n_slider]))
display(widgets.out_box)

A detailed series of tutorials showcasing feature and use-cases of `mBuild` can be found at https://github.com/mosdef-hub/mosdef_tutorials)


Due to restriction of forcefield parameters availability, the example workflow will continue with the box of compound selected from the drop down list.

### Foyer - https://github.com/mosdef-hub/foyer/

`Foyer` is the atom-typing and parameterization package within the MoSDeF suite of tools. In molecular simulation, the bonded and non-bonded parameters that define the interactions between particles is called a Force Field.  These interaction parameters are derived by fitting to a specific physical property, such as density.


In `Foyer`, force field information is contained within an XML file, that can be loaded in as a `Forcefield` object.  By calling `Forcefield.apply()` on an `mbuild.Compound`, the force field parameters are automatically applied to the compound, returning a fully parametrized system.


Below we will demonstrate the process of initializing a `Forcefield` object with foyer, and applying the force field parameters to the `mBuild.Compound` initialized above.  We will use the General Amber Force Field (GAFF) using the `GAFF-foyer` plugin, and we will assign partial charges to each atom by using the `antefoyer` plugin.

In [ ]:
import foyer
import antefoyer

gaff_ff = foyer.forcefields.load_GAFF()

if widgets.BOX_OF_COMPOUNDS:
    typed_compound = gaff_ff.apply(widgets.BOX_OF_COMPOUNDS,
                              assert_dihedral_params=False)
else:
    print('Please pick a molecule from the dropdown above.')

We have written a small function to help apply the partial charges with `antefoyer` and `antechamber`.  The charge assignment method we are using is `AM1BCC`, and we are setting the net charge to `0.0`.

In [ ]:
def apply_charges(box_structure, single_compound, n_atoms, ff):
    single_mol_struct_charge = antefoyer.ante_charges(
            single_compound, 'bcc', net_charge=0.00, multiplicity=1)
    
    for index, atom in enumerate(box_structure.atoms):
        atom.charge = single_mol_struct_charge.atoms[index%n_atoms].charge
    return box_structure

In [ ]:
charge_structure = apply_charges(box_structure=typed_compound,
                                 single_compound=widgets.COMPOUND,
                                 n_atoms=widgets.COMPOUND.n_particles,
                                 ff=gaff_ff)

### GMSO (under development) - https://github.com/mosdef-hub/gmso

`GMSO` is designed to be a general and flexible representation of chemical topologies for molecular system. The package is currently underdevelopment, but it is set to become the backend data structure for `foyer` to store all the information of a typed molecular system.

The main attribute that set `GMSO` apart from other available similar library is its generality and explicity. By assuming at little as possible about the chemical system, model, or engine, `GMSO` can support a variety of systems with different type of force field. 

In [ ]:
import gmso
import gmso.external
import gmso.formats
import warnings
warnings.filterwarnings("ignore")

topology = gmso.external.from_parmed(charge_structure)
topology.name = widgets.COMPOUND.name

`GMSO` emphasizes on the explicity of forcefield parameters. Hence, the library utilizes `sympy.expression` to store the expression for all the Type (AtomType, BondType, etc.), and `unyt` to keep track of all corresponding parameters' unit. An example of an AtomType's expression and parameters is shown below.

In [ ]:
display(topology.sites[0].atom_type.expression)
display(topology.sites[0].atom_type.parameters)

Last but not least, the most important task of `GMSO`, or any chemical topolgy backend, is to save out accurate molecular topology file that can be read and execute by simulation engine. Since generality is one of the main focus of `GMSO`, we plan to support writing out to as many types of molecular file as possible. Currently, we already have writers for a few popular simulation code such as LAMMPS and GROMACS. 

In [ ]:
gmso.formats.write_top(topology, "simulation/topol.top", 
                       top_vars={"fudgeLJ": 0.5, "fudgeQQ": 0.8, "comb-rule": "geometric"})
gmso.formats.write_gro(topology, "simulation/conf.gro")

Time to run the simulation! We are using GROMACS to simulate a short equilibrium simulation of small molecules in a box.

In [ ]:
from run_simulation import run_energy_minimization, run_nvt
run_energy_minimization()
run_nvt()

The result can be analyzed by other available analysis library such as `panedr`.

In [ ]:
import panedr
import matplotlib.pyplot as plt
sim_data = panedr.edr_to_df("simulation/ener.edr")

plt.figure(dpi=350, figsize=(5, 5))
plt.plot(sim_data["Time"], sim_data["Temperature"])

We can visualize the what happened in in the simulation using `nglview`.

In [ ]:
from run_simulation import visualize_trajectory

visualize_trajectory()

The script to run this example workflow can be found in `workflow.py`, which is in the same directory as this notebook. User can execute the script by `python workflow.py`, and follow instruction to provide:
- Path to the molecule file
- Dimension of the box 
- Number of molecules in the box

### More links

- The above example only present a fraction of MoSDeF's capabilities. More information of about this software suite can be found at:
    - https://mosdef.org
    - https://github.com/mosdef-hub
- More example of MoSDeF workflow with more interesting system can be found at:
    - https://github.com/mosdef-hub/mosdef-workflows
    - https://github.com/daico007/TRUE-nanotribology
    - https://github.com/uppittu11/true_lipids
    - https://github.com/rmatsum836/true_graphene